## Подготовка

In [583]:
import pandas as pd
import numpy as np
# читаю файл
df = pd.read_csv('test_data.csv')

In [584]:
# индексация
indx = pd.Index(range(len(df)))
df = df.set_index(indx)

In [585]:
# создаю пустой столбец
df['insight'] = np.nan

In [586]:
# делаю все буквы в текстовом столбце строчными
df['text'] = df['text'].str.lower()

## Приветствие и прощание

In [587]:
# размечаю данные с приветствием
df['insight'] = np.where(((df['text'].str.contains("здравств")) | (df['text'].str.contains("добрый день"))) & (df['role']==("manager")), 
                         "greeting=True", df["insight"])

In [588]:
# размечаю данные с прощанием
df['insight'] = np.where(((df['text'].str.contains("до свидания")) | (df['text'].str.contains("всего доброго"))) & (df['role']==("manager")), 
                         "parting=True", df["insight"])

In [589]:
# реплики с приветствием
df.loc[df['insight'] == "greeting=True"]

,dlg_id,line_n,role,text,insight
1,0,1,manager,алло здравствуйте,greeting=True
110,1,1,manager,алло здравствуйте,greeting=True
166,2,2,manager,алло здравствуйте,greeting=True
250,3,1,manager,алло дмитрий добрый день,greeting=True


In [590]:
# реплики с прощанием
df.loc[df['insight'] =="parting=True"]

,dlg_id,line_n,role,text,insight
108,0,108,manager,всего хорошего до свидания,parting=True
163,1,54,manager,до свидания,parting=True
300,3,51,manager,угу все хорошо да понедельника тогда всего доб...,parting=True
335,4,33,manager,во вторник все ну с вами да тогда до вторника ...,parting=True
479,5,142,manager,ну до свидания хорошего вечера,parting=True


In [591]:
# проверка выполнил ли менеджер требование(поздороваться и попрощаться)
# выбор строк с разметкой
test = df.loc[((df['insight'] == "greeting=True") | (df['insight'] =="parting=True"))]
# создание словаря с ключами из dlg_id
res = (test
       .groupby("dlg_id")
       .apply(lambda x: x.drop(columns="dlg_id").to_dict("records"))
       .to_dict())
# создание списка с dlg_id, в которых менеджер выполнил требование
check = []
for i in res:
  if (len(res[i]) == 2): check.append(i)
check

[0, 1, 3]

## Имя менеджера и представился ли он

In [592]:
import nltk
nltk.download('punkt')
import pymorphy2

# возвращает имя из текста, если не находит, возвращает 0
def name(text):
  # порог оценки вероятности
  prob_thresh = 0.4

  morph = pymorphy2.MorphAnalyzer()

  for word in nltk.word_tokenize(text):
    for p in morph.parse(word):
      if 'Name' in p.tag and p.score >= prob_thresh:
        return word

  return 0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [593]:
# размечаю данные с представлением менеджера
df['name_manager'] = df['text'].apply(name) #распознает имена в тексте
df['name_manager'] = np.where((((df['text'].str.contains("меня")) & (df['text'].str.contains("зовут"))) | (df['text'].str.contains("это"))) & (df['role']==("manager")) & (df['name_manager'] != 0), df['name_manager'], np.nan)



In [594]:
df

,dlg_id,line_n,role,text,insight,name_manager
0,0,0,client,алло,nan,NaN
1,0,1,manager,алло здравствуйте,greeting=True,NaN
2,0,2,client,добрый день,nan,NaN
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,nan,ангелина
4,0,4,client,ага,nan,NaN
...,...,...,...,...,...,...
475,5,138,manager,по поводу виджетов и с ними уже обсудите конкр...,nan,NaN
476,5,139,manager,все я вам высылаю счет и с вами на связи если ...,nan,NaN
477,5,140,client,спасибо спасибо,nan,NaN
478,5,141,client,да да тогда созвонимся ага спасибо вам давайте,nan,NaN


In [595]:
# имена представившихся менеджеров на каждый звонок

test = df.loc[(((df['text'].str.contains("меня")) & (df['text'].str.contains("зовут"))) | (df['text'].str.contains("это"))) & (df['role']==("manager"))]
# создание словаря с ключами из dlg_id
res = (test
       .groupby("dlg_id")
       .apply(lambda x: x.drop(columns="dlg_id").to_dict("records"))
       .to_dict())
for i in res:
  print(i, res[i][0]['name_manager'])

0 ангелина
1 ангелина
2 ангелина
3 максим
4 nan
5 анастасия


## Название компании

In [596]:
df.loc[(df['text'].str.contains("компания")) & (df['role']==("manager"))]

,dlg_id,line_n,role,text,insight,name_manager
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,nan,ангелина
111,1,2,manager,меня зовут ангелина компания диджитал бизнес з...,nan,ангелина
167,2,3,manager,меня зовут ангелина компания диджитал бизнес з...,nan,ангелина
251,3,2,manager,добрый меня максим зовут компания китобизнес у...,nan,максим
273,3,24,manager,транспортная компания хорошо а у вас уже вот в...,nan,NaN


In [597]:
# создаю датафрейм из строчек со словом "компания" в тексте
test = df.loc[(df['text'].str.contains("компания")) & (df['role']==("manager"))]

res = (test
       .groupby("dlg_id")
       .apply(lambda x: x.drop(columns="dlg_id").to_dict("records"))
       .to_dict())
# беру следующее слово после слова "компания" как название компании
# в отобранный датафрейм также попала строчка, где менеджер говорит слово компания не в нужном нам контексте, однако до этого он называл свою компанию, поэтому всё получилось корректно
for i in res:
  res[i][0]['text'] = nltk.word_tokenize(res[i][0]['text'])
  for j in range(len(res[i][0]['text'])):
    if (res[i][0]['text'][j] == 'компания'):
      res[i] = res[i][0]['text'][j+1]
      break
res

{0: 'диджитал', 1: 'диджитал', 2: 'диджитал', 3: 'китобизнес'}